In [4]:
import requests
import pandas as pd

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"
response = requests.get(url)
data = response.json()

df = pd.DataFrame(data)

df.shape

(230, 5)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Intitulé du poste    230 non-null    object
 1   Date de publication  230 non-null    object
 2   lieu                 230 non-null    object
 3   competences          230 non-null    object
 4   Type de poste        230 non-null    object
dtypes: object(5)
memory usage: 9.1+ KB


In [6]:
df.describe()

,Intitulé du poste,Date de publication,lieu,competences,Type de poste
count,230,230,230,230,230
unique,203,32,72,203,79
top,[DEVELOPPEUR AGILE-(H/F)\n],\npostée il y a 1 mois\n,[\nParis\n],"[\nagile\n, \nsql\n, \nsupport\n, \npython\n, ...","[\n, \n, Société générale, \n, Client final, \..."
freq,3,24,38,4,35


In [8]:
df.columns

Index(['Intitulé du poste', 'Date de publication', 'lieu', 'competences',
       'Type de poste'],
      dtype='object')

In [10]:
df.count()

Intitulé du poste      230
Date de publication    230
lieu                   230
competences            230
Type de poste          230
dtype: int64

In [31]:
import requests
import pandas as pd
import re

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"
response = requests.get(url)
data = response.json()

for i in range(len(data)):
    data[i]['Intitulé du poste'] = [x.strip() for x in data[i]['Intitulé du poste']]
    data[i]['Date de publication'] = data[i]['Date de publication'].strip().replace("postée ", "").replace("il y a ", "")
    data[i]['lieu'] = [x.strip() for x in data[i]['lieu']]
    data[i]['competences'] = [x.strip() for x in data[i]['competences']]
    data[i]['Type de poste'] = [x.strip() for x in data[i]['Type de poste']]

df = pd.DataFrame(data)
df


,Intitulé du poste,Date de publication,lieu,competences,Type de poste
0,[Data Analyst - Publicité],21 jours,[Paris],"[sql, t-sql, outils]","[, , Sept Lieues SAS, , Cabinet de recrutement..."
1,[Data analyst - H/F],1 mois,"[Paris, La Défense]","[support, outils, agile, design, big data]","[, , EDF France, , Client final, , , CDI - tem..."
2,[Customer Data Analyst (H/F)],27 jours,[Paris],"[crm, data quality, sql, sas, python]","[, , Hermès, , Client final, , , CDI - temps p..."
3,[Master Data Analyst H/F],21 jours,[Moussy-le-Vieux],"[si, support, sap]","[, , TEREOS, , Client final, , , CDI - temps p..."
4,[Portfolio Data Analyst H/F],3 jours,[France],"[outils, support, decisionnel, sql, test]","[, , Safran-Group, , Client final, , , CDI - t..."
...,...,...,...,...,...
225,[SAP Fiori Developer (H/F)],27 jours,[Saint-Ouen],"[sap, abap, progiciel, système, support]","[, , Groupe ENGIE, , Client final, , , CDI - t..."
226,[ENGIE Digital - Architecte Données (H/F)],3 jours,[Paris],"[digital, outils, securité, sécurité, t-sql]","[, , Groupe ENGIE, , Client final, , , CDI - t..."
227,[Quality Assurance - Lead QA - Lead Acceptance...,2 heures,[Boulogne-Billancourt],"[test, serveur, jquery, mobile, tdd]","[, , Francaise des jeux, , Client final, , , C..."
228,[DATA SCIENTIST],16 jours,"[NEUILLY SUR SEINE, 38.000,00 € - 50.000,00 € ...","[nosql, unix, java, machine learning, linux]","[, , BEEZEN, , ESN, , , CDI - temps plein]"


In [50]:
import requests
import pandas as pd

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)

df['salaire'] = df['lieu'].apply(lambda x: [i for i in x if i.find("€")!=-1])

for i in range(len(data)):
    data[i]['Intitulé du poste'] = [x.strip() for x in data[i]['Intitulé du poste']]
    data[i]['Date de publication'] = data[i]['Date de publication'].strip().replace("postée ", "").replace("il y a ", "")
    data[i]['lieu'] = [x.strip() for x in data[i]['lieu']]
    data[i]['competences'] = [x.strip() for x in data[i]['competences']]
    data[i]['Type de poste'] = [x.strip() for x in data[i]['Type de poste']]

df.head(24)


,Intitulé du poste,Date de publication,lieu,competences,Type de poste,salaire
0,[\nData Analyst - Publicité\n],\npostée il y a 21 jours\n,[\nParis\n],"[\nsql\n, \nt-sql\n, \noutils\n]","[\n, \n, Sept Lieues SAS, \n, Cabinet de recru...",[]
1,[Data analyst - H/F\n],\npostée il y a 1 mois\n,"[\nParis, La Défense\n]","[\nsupport\n, \noutils\n, \nagile\n, \ndesign\...","[\n, \n, EDF France, \n, Client final, \n, \n,...",[]
2,[Customer Data Analyst (H/F)\n],\npostée il y a 27 jours\n,[\nParis\n],"[\ncrm\n, \ndata quality\n, \nsql\n, \nsas\n, ...","[\n, \n, Hermès, \n, Client final, \n, \n, \nC...",[]
3,[Master Data Analyst H/F\n],\npostée il y a 21 jours\n,[\nMoussy-le-Vieux\n],"[\nsi\n, \nsupport\n, \nsap\n]","[\n, \n, TEREOS, \n, Client final, \n, \n, \nC...",[]
4,[Portfolio Data Analyst H/F\n],\npostée il y a 3 jours\n,[\nFrance\n],"[\noutils\n, \nsupport\n, \ndecisionnel\n, \ns...","[\n, \n, Safran-Group, \n, Client final, \n, \...",[]
5,[Data Analyst BO/Qlick Sense (h/f)\n],\npostée il y a 15 jours\n,"[\nMaisons Alfort\n, \n45.000,00 € - 50.000,00...","[\nsupport\n, \nsql\n, \ndata management\n, \n...","[\n, \n, FED IT, \n, Cabinet de recrutement, \...","[\n45.000,00 € - 50.000,00 € / an\n]"
6,[Product Data Analyst - BI - H/F\n],\npostée il y a 28 jours\n,[\nParis\n],"[\ncloud\n, \npaas\n, \nqlik\n, \niaas\n, \nsu...","[\n, \n, Cegid, \n, Client final, \n, \n, \nCD...",[]
7,[Data Analyst / Chef de Projet BI (h/f)\n],\npostée il y a 1 mois\n,"[\nPARIS\n, \n60.000,00 € - 62.000,00 € / an\n]","[\ndata management\n, \nbusiness intelligence\...","[\n, \n, FED IT, \n, Cabinet de recrutement, \...","[\n60.000,00 € - 62.000,00 € / an\n]"
8,[Business analyst senior DATA CRM - Paris H/F\n],\npostée il y a 12 heures\n,[\nParis\n],"[\ncrm\n, \nagile\n, \ndecisionnel\n, \nréseau...","[\n, \n, Credit Agricole, \n, Client final, \n...",[]
9,[Data scientist / Analyst and software develop...,\npostée il y a 13 jours\n,[\nRueil-Malmaison\n],"[\nbig data\n, \nscrum\n, \nintelligence artif...","[\n, \n, Vinci, \n, Client final, \n, \n, \nSt...",[]
